In [ ]:
%%capture
import ROOT
import glob
import math
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.utility import *
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kFatal
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

In [ ]:
def drawQuantileProjection(objectName,rootDataFile,quantileOrder=0.5, axis="x"):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    quants = []
    canvas.cd()
    ROOT.gPad.SetGridy(1)
    for i,histo in enumerate(hist):
        if axis == "y":
            quant = copy(histo.QuantilesY(quantileOrder))
        else:
            quant = copy(histo.QuantilesX(quantileOrder))
        quant.SetYTitle("Median "+ histo.GetYaxis().GetTitle())
        quant.SetXTitle(histo.GetXaxis().GetTitle())
        quant.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        quant.SetLineWidth(2)
        quants.append(quant)
        quants[i].SetStats(0)
        quants[i].Draw("SAME L")
        legend.AddEntry(quant,runList[i])
    return quants,legend,canvas
    
def sliceAndFit(objectName,rootDataFile,fitFunc="gaus",fitRange=[40,60]):
    from copy import copy
    canvas = ROOT.TCanvas()
    legend = ROOT.TLegend()
    [hist,leg,canvo,pad1] = drawHistograms(objectName,rootDataFile,normalize=False,legend=False,
                                               legendNames=runList,pads=True,drawOption="COLZ",maxColumns=3)
    fits = []
    myFunc = ROOT.TF1("myFunc","gaus",40,60)
    canvas.cd()
    for i,histo in enumerate(hist):
        try:
            histo.FitSlicesY(myFunc, 0, -1, 0, "QNR")
            fit = copy(ROOT.gDirectory.Get(histo.GetName()+"_1"))
        except:
            print("Could not perform fit.")
            return [],legend,canvas
        fit.SetYTitle("Gaus Fit Mean "+ histo.GetYaxis().GetTitle())
        fit.SetXTitle(histo.GetXaxis().GetTitle())
        fit.SetTitle(quant.GetYaxis().GetTitle()+" vs. "+quant.GetXaxis().GetTitle())
        fit.SetLineWidth(2)
        fits.append(fit)
        legend.AddEntry(fit,runList[i])
        fit.Draw("SAME L")
    legend.Draw()
    return fits, legend, canvas

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetPalette(55)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
cols = list(ROOT.TColor.GetPalette())
def updateColors(histograms,palette):
    colors = []
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            color = palette[math.floor((i/(len(histograms)-1))*(len(palette)-1))]
        else:
            color = palette[0]
        #color = i
        colors.append(color)
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
    return colors
            
def setTrendRanges(trends):
    if type(trends[0]) in  (ROOT.TH1D, ROOT.TH1F, ROOT.TH1C):
        values = [binValue for trend in trends for binValue in trend if binValue != 0]
        maxRange = max(values)
        minRange = min(values)
        diff = maxRange-minRange
        for i,trend in enumerate(trends):
            if len(trends)>1:
                trend.SetMaximum(maxRange+diff/2)
                trend.SetMinimum(minRange-diff/2)
    else:
        raise Exception("Expected TH1s")

In [ ]:
# Notebook variables
periodName = "myPeriod"
passName = "myPass"
runNumber = 123456
path = "myPath"

#periodName = "LHC23zzh"
#passName = "apass4_test"
#runNumber = 123456
#path = "/cave/alice-tpc-qc/data/2023/"

# Get Passes to compare (latest 2)
search_dir = path+"/"+periodName+"/"
folders = glob.glob(search_dir + "*")
folders = [folder for folder in folders if ("." not in folder)]
def orderPasses(apass):
    if "pass" not in apass:
        return -1
    result = int(apass.split("_")[0].split("pass")[1])
    if "_" in apass:
        if "old" in apass.split("_")[1]:
            result -= 1/sum([ord(c) for c in apass.split("_")[1]])
        else:
            result -= 1/sum([ord(c) for c in apass.split("_")[1]])
    passType = apass.split("_")[0].split("pass")[0]
    if passType == "a":
        result += 100
    return result

folders.sort(key=lambda x: orderPasses(x.split("/")[-1]))
folders = [folder for folder in folders if (orderPasses(folder.split("/")[-1])<orderPasses(passName))]

if not folders:
    raise Exception("At least 2 passes are needed to compare!")


# Automatically get the previous pass for comparison
passName_comparison = folders[-1].split("/")[-1]

# Set manually which pass should be used for comparison
#passName_comparison = "apass1"

runList = [passName,passName_comparison]

# Find runs
fileList = glob.glob(path+"/"+periodName+"/"+passName+"/"+"*_QC.root")
fileList = [file for file in fileList if file[-13] != "_"]
fileList.sort()
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
#print(runList)
# Find runs in period to be compared
fileList_comparison = glob.glob(path+"/"+periodName+"/"+passName_comparison+"/"+"*_QC.root")
fileList_comparison = [file for file in fileList_comparison if file[-13] != "_"]
fileList_comparison.sort()
runList_comparison = [fileList_comparison[i][-14:-8] for i in range(len(fileList_comparison))]

# Make sure only the same runs are there
runList_temp = [run for run in runList if run in runList_comparison]
runList = runList_temp
fileList = [file for file in fileList if file[-14:-8] in runList]
fileList_comparison = [fileList_comparison[i] for i in range(len(fileList_comparison)) if runList_comparison[i] in runList]
runList_comparison = runList

# Read the Root Files
rootDataFile = [ROOT.TFile.Open(file,"READ") for file in fileList]   
rootDataFile_comparison = [ROOT.TFile.Open(file,"READ") for file in fileList_comparison]

In [ ]:
display(Markdown(f"# TPC Async QC Report - {periodName} - {passName} vs {passName_comparison}"))

**[RCT Table](https://ali-bookkeeping.cern.ch/?page=lhc-period-overview)**

In [ ]:
import datetime
now = datetime.datetime.now()
display(Markdown("Report automatically created on **"+now.strftime("%d.%m.%Y @ %H:%M:%S")+"**"))

In [ ]:
runList_markdown = ""
for run in runList:
    runList_markdown += f"`{run}` "

display(Markdown("**Runs in comparison:** \n\n"+runList_markdown))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Trendings" data-toc-modified-id="Trendings-1">Trendings</a></span><ul class="toc-item"><li><span><a href="#Number-of-tracks" data-toc-modified-id="Number-of-tracks-1.1">Number of tracks</a></span></li><li><span><a href="#Mean-phi-of-tracks" data-toc-modified-id="Mean-phi-of-tracks-1.2">Mean phi of tracks</a></span><ul class="toc-item"><li><span><a href="#A-Side" data-toc-modified-id="A-Side-1.2.1">A-Side</a></span></li><li><span><a href="#C-Side" data-toc-modified-id="C-Side-1.2.2">C-Side</a></span></li></ul></li><li><span><a href="#Mean-eta-of-tracks" data-toc-modified-id="Mean-eta-of-tracks-1.3">Mean eta of tracks</a></span></li><li><span><a href="#Mean-tracks-transverse-momentum-of-tracks" data-toc-modified-id="Mean-tracks-transverse-momentum-of-tracks-1.4">Mean tracks transverse momentum of tracks</a></span></li><li><span><a href="#Mean-charge-sign-of-tracks" data-toc-modified-id="Mean-charge-sign-of-tracks-1.5">Mean charge sign of tracks</a></span></li><li><span><a href="#Mean-dEdxTot-of-MIPs-tracks" data-toc-modified-id="Mean-dEdxTot-of-MIPs-tracks-1.6">Mean dEdxTot of MIPs tracks</a></span></li></ul></li><li><span><a href="#Plots" data-toc-modified-id="Plots-2">Plots</a></span><ul class="toc-item"><li><span><a href="#Tracks" data-toc-modified-id="Tracks-2.1">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions-of-tracks" data-toc-modified-id="Geometrical-distributions-of-tracks-2.1.1">Geometrical distributions of tracks</a></span></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-2.1.2">Track properties</a></span></li></ul></li><li><span><a href="#DCAr" data-toc-modified-id="DCAr-2.2">DCAr</a></span></li><li><span><a href="#DCAr-(with-pT-cut-(>1-GeV))" data-toc-modified-id="DCAr-(with-pT-cut-(>1-GeV))-2.3">DCAr (with pT cut (&gt;1 GeV))</a></span></li><li><span><a href="#PID" data-toc-modified-id="PID-2.4">PID</a></span><ul class="toc-item"><li><span><a href="#dEdxTot-of-MIP" data-toc-modified-id="dEdxTot-of-MIP-2.4.1">dEdxTot of MIP</a></span></li><li><span><a href="#TPC-Gain-calibration" data-toc-modified-id="TPC-Gain-calibration-2.4.2">TPC Gain calibration</a></span></li><li><span><a href="#dEdxMax-of-MIP" data-toc-modified-id="dEdxMax-of-MIP-2.4.3">dEdxMax of MIP</a></span></li></ul></li></ul></li></ul></div>

---
## Trendings

### Number of tracks

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
trending = "entries"
error = ""
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",drawOption="SAME L P E")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas,drawOption="SAME L P E")
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
trend.SetTitle("Number of tracks")
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.SetCanvasSize(1200,400)
canvas.Draw()

### Mean phi of tracks

#### A-Side

In [ ]:
%jsroot on
objectName="hPhiAside"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.SetCanvasSize(1200,400)
canvas.Draw()

#### C-Side

In [ ]:
%jsroot on
objectName="hPhiCside"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean eta of tracks

In [ ]:
%jsroot on
objectName="hEta;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean tracks transverse momentum of tracks

In [ ]:
%jsroot on
objectName="hPt;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean charge sign of tracks

In [ ]:
%jsroot on
objectName="hSign;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
#trend.SetTitle("Number of tracks - Trend")
canvas.SetCanvasSize(1000,400)
canvas.Draw()

### Mean dEdxTot of MIPs tracks

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
trending = "fit(gaus,Sq,0,40,60)"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
canvas.cd()
trend.Draw("L P E")
trend_comparison.Draw("SAME L P E")
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.Draw()

**Mean number of clusters per tracks:**

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean tan(lambda) of tracks:**

In [ ]:
%jsroot on
objectName="hdEdxVsTgl;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.SetCanvasSize(1000,400)
canvas.Draw()

**Mean sin(phi) of tracks:**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC;1"
trending = "mean"
error = "meanError"
leg = ROOT.TLegend()
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none")
[trend_comparison,canvas] = drawTrending(objectName,rootDataFile_comparison,names=runList,namesFromRunList=True,
                               trend=trending,error=error,axis=1,log="none",canvas=canvas)
setTrendRanges([trend,trend_comparison])
updateColors([trend,trend_comparison],palette=[ROOT.kBlue+3,ROOT.kRed-3])
leg.AddEntry(trend,passName)
leg.AddEntry(trend_comparison,passName_comparison)
leg.Draw()
canvas.SetCanvasSize(1000,400)
canvas.Draw()

---
## Plots

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetPalette(57)

---
### Tracks
#### Geometrical distributions of tracks
**Phi distribution A-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiAside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",yAxisRange=[2.2e-3,3.2e-3],
                                                                      grid=True)
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

**Phi distribution C-side tracks:**

In [ ]:
%jsroot on
objectName="hPhiCside"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",yAxisRange=[2.2e-3,3.2e-3],
                                                                      grid=True)
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

**Eta distribution:**

In [ ]:
%jsroot on
objectName="hEta"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",xAxisRange=[-1.1,1.1],
                                                                      grid=True)
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName+" - Median eta vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName_comparison+" - Median eta vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
c.Draw()

#### Track properties
**Transverse momentum**

In [ ]:
%jsroot on
objectName="hPt"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",log="logxy",
                                                                      grid=True)
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

**Charge / transverse momentum**

In [ ]:
%jsroot on
objectName="hQOverPt"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",
                                                                      grid=True)
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

**Number of clusters per track**

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",grid=True,
                                                                      xAxisRange = [50,160],yAxisRange=[0,40e-3])
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="crossedRows"
if checkIfExists(rootDataFile,objectName) and checkIfExists(rootDataFile_comparison,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",grid=True,
                                                                      xAxisRange = [50,160])
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="sharedClusters"
if checkIfExists(rootDataFile,objectName) and checkIfExists(rootDataFile_comparison,objectName):
    [hist,legend,canvas,pad1,histosComp,histosRatio] = drawHistograms(objectName,rootDataFile,compareTo=rootDataFile_comparison,
                                                                      normalize=True,legend=False,legendNames=runList,pads=True,
                                                                      maxColumns=2,drawOption="SAME L",grid=True,
                                                                      xAxisRange = [0,160],log="logy")
    leg = ROOT.TLegend()
    leg.AddEntry(hist[0],passName)
    leg.AddEntry(histosComp[0],passName_comparison)
    leg.Draw()
    canvas.Draw()

**nClusters vs phi**

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- A-Side - nClusters per track vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName_comparison+"- A-Side - nClusters per track vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
c.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- C-Side - nClusters per track vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName_comparison+"- C-Side - nClusters per track vs phi")
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
c.Draw()

**nClusters vs eta**

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGrid(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetXaxis().SetRangeUser(-1.1,1.1)
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- nClusters per track vs eta")
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGrid(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetXaxis().SetRangeUser(-1.1,1.1)
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName_comparison+"- nClusters per track vs eta")
    legend_comparison.Draw()
c.Draw()

**nClusters vs transverse momentum**

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGrid(1)
    canvas.SetLogx()
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName+"- nClusters per track vs pT")
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGrid(1)
    canvas_comparison.SetLogx()
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(60,160)
        quant.SetTitle(passName_comparison+"- nClusters per track vs pT")
    legend_comparison.Draw()
c.Draw()

### DCAr

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos"
title = " - DCAr A-Side - Positive"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg"
title = " - DCAr A-Side - Negative"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos"
title = "DCAr C-Side - Positive"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg"
title = "DCAr C-Side - Negative"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

### DCAr (with pT cut (>1 GeV))

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos_pTmin"
title = " - DCAr A-Side - Positive - pT > 1 Gev"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg_pTmin"
title = " - DCAr A-Side - Negative - pT > 1 Gev"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos_pTmin"
title = "DCAr C-Side - Positive - pT > 1 Gev"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg_pTmin"
title = "DCAr C-Side - Negative - pT > 1 Gev"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = drawQuantileProjection(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = drawQuantileProjection(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(-3,3)
        quant.SetTitle(passName_comparison+title)
    sectorBoundary = ROOT.TLine(0,quant.GetMinimum(),0,quant.GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,quant.GetMinimum(),(j*2*math.pi)/18,quant.GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,quant.GetMaximum()*0.95+quant.GetMinimum()*0.05,str(j))
    legend_comparison.Draw()
    c.Draw()

### PID
#### dEdxTot of MIP

In [ ]:
%jsroot on
yRange = [0,0.09]
objects = ["hdEdxTotMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=yRange,log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    for hist in hists[0]:
        hist.SetTitle(passName+" - "+hist.GetTitle())
        hist.Draw("SAME L")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        for hist in hists[i]:
            hist.SetTitle(passName+" - "+hist.GetTitle())
            hist.Draw("SAME L")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

In [ ]:
%jsroot on
yRange = [0,0.09]
objects = ["hdEdxTotMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile_comparison,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile_comparison,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=yRange,log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    for hist in hists[0]:
        hist.SetTitle(passName_comparison+" - "+hist.GetTitle())
        hist.Draw("SAME L")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        for hist in hists[i]:
            hist.SetTitle(passName_comparison+" - "+hist.GetTitle())
            hist.Draw("SAME L")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

#### TPC Gain calibration
**dEdx of MIP vs sector**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
title = "TPC Mean MIP Q_{Tot} vs Sector"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

**dEdx of Mip vs nClusters**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
title = "TPC Mean MIP Q_{Tot} vs nClusters"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(60,155)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(60,155)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

**dEdx of Mip vs tan(lambda)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
title = "TPC Mean MIP Q_{Tot} vs tan(lambda)"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-2,2)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-2,2)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

**dEdx of Mip vs sin(phi)**

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
title = "TPC Mean MIP Q_{Tot} vs sin(phi)"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

#### dEdxMax of MIP

In [ ]:
%jsroot on
yRange = [0,0.09]
objects = ["hdEdxMaxMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=yRange,log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    for hist in hists[0]:
        hist.SetTitle(passName+" - "+hist.GetTitle())
        hist.Draw("SAME L")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        for hist in hists[i]:
            hist.SetTitle(passName+" - "+hist.GetTitle())
            hist.Draw("SAME L")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

In [ ]:
%jsroot on
yRange = [0,0.09]
objects = ["hdEdxMaxMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile_comparison,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile_comparison,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",yAxisRange=yRange,log="logx")
        updateColors(hist,cols)
        hists.append(copy(hist))
    c = ROOT.TCanvas("MIP","MIP",1100,600)
    c.Divide(2)
    leftPad = c.cd(1)
    leftPad.SetGrid(1)
    for hist in hists[0]:
        hist.SetTitle(passName_comparison+" - "+hist.GetTitle())
        hist.Draw("SAME L")
        hist.SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,yRange[0],50,yRange[1]) for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
    targets[0].Draw()
    legend.Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        for hist in hists[i]:
            hist.SetTitle(passName_comparison+" - "+hist.GetTitle())
            hist.Draw("SAME L")
            hist.SetStats(0)
        targets[i].Draw()
    c.Draw()

**dEdx of MIP vs sector**

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsSec_TPC"
title = "TPC Mean MIP Q_{Max} vs Sector"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

**dEdx of Mip vs nClusters**

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsNcl_TPC"
title = "TPC Mean MIP Q_{Max} vs nClusters"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(60,155)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(60,155)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

**dEdx of Mip vs tan(lambda)**

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsTgl_TPC"
title = "TPC Mean MIP Q_{Max} vs tan(lambda)"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-2,2)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-2,2)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()

**dEdx of Mip vs sin(phi)**

In [ ]:
%jsroot on
objectName="hdEdxMaxMIPVsSnp_TPC"
title = "TPC Mean MIP Q_{Max} vs sin(phi)"
if checkIfExists(rootDataFile,objectName):
    c = ROOT.TCanvas("","",1200,500)
    c.Divide(2)
    [quants,legend,canvas] = sliceAndFit(objectName,rootDataFile)
    updateColors(quants,cols)
    canvas = c.cd(1)
    canvas.SetGridy(1)
    for quant in quants:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName+" - "+title)
    legend.Draw()
    
if checkIfExists(rootDataFile_comparison,objectName):
    [quants_comparison,legend_comparison,canvas_comparison] = sliceAndFit(objectName,rootDataFile_comparison)
    updateColors(quants_comparison,cols)
    canvas_comparison = c.cd(2)
    canvas_comparison.SetGridy(1)
    for quant in quants_comparison:
        quant.Draw("L SAME")
        quant.GetYaxis().SetRangeUser(45,55)
        quant.GetXaxis().SetRangeUser(-1,1)
        quant.SetTitle(passName_comparison+" - "+title)
    legend_comparison.Draw()
c.Draw()